If you're opening this Notebook on colab, you will probably need to install 🤗 Transformers and 🤗 Datasets. Uncomment the following cell and run it.

In [ ]:
!pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 108.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.4 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 18.6 MB/s eta 0:00:00


If you're opening this notebook locally, make sure your environment has an install from the last version of those libraries.

To be able to share your model with the community and generate results like the one shown in the picture below via the inference API, there are a few more steps to follow.

First you have to store your authentication token from the Hugging Face website (sign up [here](https://huggingface.co/join) if you haven't already!) then execute the following cell and input your username and password:

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.9.2-1).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.7/211.7 kB 30.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=363010b03300a6af55729e7b288576022985944a842d9e355776f3129475dcd2
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [ ]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.30.2


You can find a script version of this notebook to fine-tune your model in a distributed fashion using multiple GPUs or TPUs [here](https://github.com/huggingface/transformers/tree/master/examples/language-modeling).

We also quickly upload some telemetry - this tells us which examples and software versions are getting used so we know where to prioritize our maintenance efforts. We don't collect (or care about) any personally identifiable information, but if you'd prefer not to be counted, feel free to skip this step or delete this cell entirely.

In [ ]:
from transformers.utils import send_example_telemetry

send_example_telemetry("language_modeling_from_scratch_notebook", framework="pytorch")

# Train a language model

In this notebook, we'll see how to train a [🤗 Transformers](https://github.com/huggingface/transformers) model on a language modeling task. We will cover two types of language modeling tasks which are:

- Causal language modeling: the model has to predict the next token in the sentence (so the labels are the same as the inputs shifted to the right). To make sure the model does not cheat, it gets an attention mask that will prevent it to access the tokens after token i when trying to predict the token i+1 in the sentence.

![Widget inference representing the causal language modeling task](https://github.com/huggingface/notebooks/blob/main/examples/images/causal_language_modeling.png?raw=1)

- Masked language modeling: the model has to predict some tokens that are masked in the input. It still has access to the whole sentence, so it can use the tokens before and after the tokens masked to predict their value.

We will see how to easily load and preprocess the dataset for each one of those tasks, and how to use the `Trainer` API to train a model on it.

This notebooks assumes you have trained a tokenizer on the corpus you are using, see the [How to train a tokenizer](https://github.com/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb) notebook ([open in colab](https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/tokenizer_training.ipynb)).

A script version of this notebook you can directly run on a distributed environment or on TPU is available in our [examples folder](https://github.com/huggingface/transformers/tree/master/examples).

## Preparing the dataset

For each of those tasks, we will use the [Wikitext 2]() dataset as an example. You can load it very easily with the 🤗 Datasets library.

In [ ]:
from datasets import load_dataset
datasets = load_dataset("muhtasham/tj-corpus-poem", split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/muhtasham___text/muhtasham--tj-corpus-poem-8385e14856123b85/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


In [ ]:
datasets

Dataset({
    features: ['text'],
    num_rows: 1438120
})

In [ ]:
# Split the dataset into train and validation sets
datasets = datasets.train_test_split(test_size=0.1)  # Adjust test_size as needed

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1294308
    })
    test: Dataset({
        features: ['text'],
        num_rows: 143812
    })
})

You can replace the dataset above with any dataset hosted on [the hub](https://huggingface.co/datasets) or use your own files. Just uncomment the following cell and replace the paths with values that will lead to your files:

In [ ]:
# datasets = load_dataset("text", data_files={"train": path_to_train.txt, "validation": path_to_validation.txt}

You can also load datasets from a csv or a JSON file, see the [full documentation](https://huggingface.co/docs/datasets/loading_datasets.html#from-local-files) for more information.

To access an actual element, you need to select a split first, then give an index:

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [ ]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,text
0,"«Рақсу ҷавлон бар сари майдон кунанд,"
1,"Имрӯз кист, ёди фақирон кунад зи дур?"
2,"Чун кўзањои бешароб,"
3,"Ки «Тахти ду фарзанди худро бигир,"
4,
5,Гурозон биёмад ба шаҳри Ситахр.
6,"Мулло Шокир Сиёҳмушкинро савор шуда аз дарвоза баромад, танҳо баъди он Садафмоҳ тавонист, ки азмашро ба ҷо орад. Ӯ низ сӯи Душанбе роҳӣ шуд ва баъди се рӯз баргашт. Чӣ аҷобате буд, ки Мулло Шокир дар ин се рӯз аз паи ҷустуҷӯи вай нашуд, ӯ кори заруре дошт ва ё набудани Садафмоҳ барояш беҳтар буд, вагарна як соат Садафмоҳ аз наздаш дур шавад, ҳазор бор аз дигарон мепурсид ва ё аз паи ҷустуҷӯяш меафтид. Садафмоҳ низ аз чунин рафтори шавҳар ҳайрон. Вай гумон кард, ки баробари рӯ ба рӯ шудан шавҳар сахт гуфта дили вайро ранҷонад ва ӯ худдорӣ накарда, ноҷо гуфта, бо ҳамин ба зиндагиашон нуқта гузоранд. Вале вакте, ки Садафмоҳ аз камарбанди зарринаш чанд тангаи тилло бароварда ба шавҳар дароз кард, ӯ чизе нагуфт:"
7,"гирифт: «Биҷаҳ!» Писар ҳайрон буд, ки чӣ гуна биҷаҳад. – «Ана ин хел! Вазн бар пой биандозу биҷаҳ!» – Писар чунон кард, ки модар фармуд. Ҳар ду бар ҳаво шуданд ва бар тахт нишастанд. Модар кӯдакро тахтнишинӣ омӯхт ва худ ба он ҷонибе, ки уқоби сиёҳ парвоз карда буд, бол кушод."
8,"Чун шеър бихонданд, ӯ олим набуд, андар наёфт. Муҳаммад ибни Васиф ҳозир буд ва дабири расоили ӯ буд ва адабӣ нек ӯ донист ва ба он рӯзгор номаи порсӣ набуд. Пас Яъқуб гуфт: ""Чизе, ки ман андар наёбам, чаро бояд гуфт?"
9,Гирди сандуқ чу мор бозӣ кун.


As we can see, some of the texts are a full paragraph of a Wikipedia article while others are just titles or empty lines.

## Causal Language modeling

For causal language modeling (CLM) we are going to take all the texts in our dataset and concatenate them after they are tokenized. Then we will split them in examples of a certain sequence length. This way the model will receive chunks of contiguous text that may look like:
```
part of text 1
```
or
```
end of text 1 [BOS_TOKEN] beginning of text 2
```
depending on whether they span over several of the original texts in the dataset or not. The labels will be the same as the inputs, shifted to the left.

We will use the [`gpt2`](https://huggingface.co/gpt2) architecture for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=causal-lm) instead. For the tokenizer, you can replace the checkpoint by the one you trained yourself.

In [ ]:
model_checkpoint = "gpt2"
tokenizer_checkpoint = "muhtasham/gpt2-tajik"

To tokenize all our texts with the same vocabulary that was used when training the model, we have to download a pretrained tokenizer. This is all done by the `AutoTokenizer` class:

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)

We can now call the tokenizer on all our texts. This is very simple, using the [`map`](https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map) method from the Datasets library. First we define a function that call the tokenizer on our texts:

In [ ]:
max_length = tokenizer.model_max_length

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=max_length)

Then we apply it to all the splits in our `datasets` object, using `batched=True` and 4 processes to speed up the preprocessing. We won't need the `text` column afterward, so we discard it.

In [ ]:
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

Map (num_proc=4):   0%|          | 0/1294308 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/143812 [00:00<?, ? examples/s]

If we now look at an element of our datasets, we will see the text have been replaced by the `input_ids` the model will need:

In [ ]:
tokenized_datasets["train"][1]

{'input_ids': [374, 6463, 12951, 12, 10739, 296, 796, 14],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

Now for the harder part: we need to concatenate all our texts together then split the result in small chunks of a certain `block_size`. To do this, we will use the `map` method again, with the option `batched=True`. This option actually lets us change the number of examples in the datasets by returning a different number of examples than we got. This way, we can create our new samples from a batch of examples.

First, we grab the maximum length our model was pretrained with. This might be a big too big to fit in your GPU RAM, so here we take a bit less at just 128.

In [ ]:
block_size = tokenizer.model_max_length
#block_size = 128

In [ ]:
block_size

1024

Then we write the preprocessing function that will group our texts:

In [ ]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

First note that we duplicate the inputs for our labels. This is because the model of the 🤗 Transformers library apply the shifting to the right, so we don't need to do it manually.

Also note that by default, the `map` method will send a batch of 1,000 examples to be treated by the preprocessing function. So here, we will drop the remainder to make the concatenated tokenized texts a multiple of `block_size` every 1,000 examples. You can adjust this behavior by passing a higher batch size (which will also be processed slower). You can also speed-up the preprocessing by using multiprocessing:

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/1294308 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/143812 [00:00<?, ? examples/s]

And we can check our datasets have changed: now the samples contain chunks of `block_size` contiguous tokens, potentially spanning over several of our original texts.

In [ ]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

' бубандад ҳаме бо сипоҳКи ҷон он ҷо расад боре, агар монад бадан бе ӯ.Аз хоҷагии ду олам озод.Аммо хушбахтона, ё мутаассифона, маро ба кор қабул карданд ва банда дар саҳрои нопайдоканори кору зиндагӣ мустақилона дар набард шудам.ҳаме хост Доро пас аз сорвонБиёвард зан хону бинҳод ростПарда.Ба забони фасеҳу лафзи матин,– Ҳасад ба чӣ расондашон? Гап занед!Он ба чанголи мани бедилудин мебоист.Ахгаре чашм ба хокистари худ дӯхтааст.Мебарӣ аз пасат ба кӯи мурод,Мо ба чаман меравем, азми тамошо кирост?!Зи некию номи накӯ соз ганҷ!Ҳарчанд дар миёнаи арзу самостам,Як вафо меору мебар сад ҳазор, Ба ман ҳар лаҳзааш то ҳол гӯёҲавои гардиши чашме рабуда-ст ихтиёрамро,Бигуфт, он чи дид аз карон то карон.Муҳаммад ибни Васиф - дабири Яъқуб ибни Лайси Саффорӣ то ҳудуди солҳои 296-297 ҳиҷрӣ (908-910 милодӣ) дар қайди ҳаёт будааст. Мувофиқи ривояти китоби «Таърихи Систон» Муҳаммад ибни Васиф шеъри арабӣ мегуфт ва барои Яъқуб ибни Лайс бори аввал ба тақлиди қасоиди арабӣ шеъри форсӣ эҷод кард. Муаллифи н

Now that the data has been cleaned, we're ready to instantiate our `Trainer`. First we create the model using the same config as our checkpoint, but initialized with random weights:

In [ ]:
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_config(config)

And we will needsome `TrainingArguments`:

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
training_args = TrainingArguments(
    output_dir=f"{model_checkpoint}-tajik",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    report_to="wandb",  # enable logging to W&B
    run_name=f"{model_checkpoint}-tajik"  # name of the W&B run (optional)
)

The last argument to setup everything so we can push the model to the [Hub](https://huggingface.co/models) regularly during training. Remove it if you didn't follow the installation steps at the top of the notebook. If you want to save your model locally in a name that is different than the name of the repository it will be pushed, or if you want to push your model under an organization and not your name space, use the `hub_model_id` argument to set the repo name (it needs to be the full name, including your namespace: for instance `"sgugger/gpt-finetuned-wikitext2"` or `"huggingface/gpt-finetuned-wikitext2"`).

We pass along all of those to the `Trainer` class:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

Cloning https://huggingface.co/muhtasham/gpt2-tajik into local empty directory.


And we can train our model:

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: muhtasham. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
1,7.110700,6.954679
2,6.701200,6.608582
3,6.546700,6.507633


TrainOutput(global_step=7215, training_loss=6.91835748007623, metrics={'train_runtime': 4965.8836, 'train_samples_per_second': 11.623, 'train_steps_per_second': 1.453, 'total_flos': 3.0161984421888e+16, 'train_loss': 6.91835748007623, 'epoch': 3.0})

Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [ ]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/475M [00:00<?, ?B/s]

To https://huggingface.co/muhtasham/gpt2-tajik
   50795d0..7850bdd  main -> main

   50795d0..7850bdd  main -> main

To https://huggingface.co/muhtasham/gpt2-tajik
   7850bdd..7e75588  main -> main

   7850bdd..7e75588  main -> main



'https://huggingface.co/muhtasham/gpt2-tajik/commit/7850bdd013a6bf87afc73b414a11aee4fe8837a1'

In [ ]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 670.24


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("sgugger/my-awesome-model")
```

## Masked language modeling

For masked language modeling (MLM) we are going to use the same preprocessing as before for our dataset with one additional step: we will randomly mask some tokens (by replacing them by `[MASK]`) and the labels will be adjusted to only include the masked tokens (we don't have to predict the non-masked tokens). If you use a tokenizer you trained yourself, make sure the `[MASK]` token is among the special tokens you passed during training!

We will use the [`bert-base-cased`](https://huggingface.co/bert-based-cased) model for this example. You can pick any of the checkpoints listed [here](https://huggingface.co/models?filter=masked-lm) instead. For the tokenizer, replace the checkpoint by the one you trained.

In [ ]:
model_checkpoint = "bert-base-cased"
tokenizer_checkpoint = "sgugger/bert-like-tokenizer"

We can apply the same tokenization function as before, we just need to update our tokenizer to use the checkpoint we just picked:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_checkpoint)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /home/sgugger/.cache/huggingface/transformers/tmpj0hlre6a


storing https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/tokenizer_config.json in cache at /home/sgugger/.cache/huggingface/transformers/8e2bf16fe90bdd2f922f692d28792f71927c19b3ec57cf1457567cd848515842.0bbe47aa0e39b09ed05a95f7d42a27299232ce8e9ef28608e8f8a1cb57a74c0a
creating metadata file for /home/sgugger/.cache/huggingface/transformers/8e2bf16fe90bdd2f922f692d28792f71927c19b3ec57cf1457567cd848515842.0bbe47aa0e39b09ed05a95f7d42a27299232ce8e9ef28608e8f8a1cb57a74c0a
https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/vocab.txt not found in cache or force_download set to True, downloading to /home/sgugger/.cache/huggingface/transformers/tmpceo1r0j0


storing https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/vocab.txt in cache at /home/sgugger/.cache/huggingface/transformers/d2c888a76d867b3c110720b637a5958d9748e157d245806d646a7c015a393b95.7b1a250944f9d3669d4f57ab97b1f6a21a494a84cff113694cf15d673e7bb6d5
creating metadata file for /home/sgugger/.cache/huggingface/transformers/d2c888a76d867b3c110720b637a5958d9748e157d245806d646a7c015a393b95.7b1a250944f9d3669d4f57ab97b1f6a21a494a84cff113694cf15d673e7bb6d5


https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /home/sgugger/.cache/huggingface/transformers/tmp9i7md8wn


storing https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/tokenizer.json in cache at /home/sgugger/.cache/huggingface/transformers/f385664afe38b7f081454cb8d1aa1ff0c4e16507dfbafe1fad8fc85538262261.159bf1d23803f236f036667eafea5ec2d6b22bcaf4b88d1dafeb8e6c7c35d6f9
creating metadata file for /home/sgugger/.cache/huggingface/transformers/f385664afe38b7f081454cb8d1aa1ff0c4e16507dfbafe1fad8fc85538262261.159bf1d23803f236f036667eafea5ec2d6b22bcaf4b88d1dafeb8e6c7c35d6f9


https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/special_tokens_map.json not found in cache or force_download set to True, downloading to /home/sgugger/.cache/huggingface/transformers/tmp58nljjm6


storing https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/special_tokens_map.json in cache at /home/sgugger/.cache/huggingface/transformers/c90d08712ec5954e9bf3853df3bab5a6549e408225f4e9b6dd90f3bc3d3c0461.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
creating metadata file for /home/sgugger/.cache/huggingface/transformers/c90d08712ec5954e9bf3853df3bab5a6549e408225f4e9b6dd90f3bc3d3c0461.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/vocab.txt from cache at /home/sgugger/.cache/huggingface/transformers/d2c888a76d867b3c110720b637a5958d9748e157d245806d646a7c015a393b95.7b1a250944f9d3669d4f57ab97b1f6a21a494a84cff113694cf15d673e7bb6d5
loading file https://huggingface.co/sgugger/bert-like-tokenizer/resolve/main/tokenizer.json from cache at /home/sgugger/.cache/huggingface/transformers/f385664afe38b7f081454cb8d1aa1ff0c4e16507dfbafe1fad8fc85538262261.159bf1d23803f236f03

Token indices sequence length is longer than the specified maximum sequence length for this model (571 > 512). Running this sequence through the model will result in indexing errors


Token indices sequence length is longer than the specified maximum sequence length for this model (554 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (522 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (657 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


And like before, we group texts together and chunk them in samples of length `block_size`. You can skip that step if your dataset is composed of individual sentences.

In [ ]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

The rest is very similar to what we had, with two exceptions. First we use a model suitable for masked LM:

In [ ]:
from transformers import AutoConfig, AutoModelForMaskedLM

config = AutoConfig.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_config(config)

loading configuration file https://huggingface.co/bert-base-cased/resolve/main/config.json from cache at /home/sgugger/.cache/huggingface/transformers/a803e0468a8fe090683bdc453f4fac622804f49de86d7cecaee92365d4a0f829.a64a22196690e0e82ead56f388a3ef3a50de93335926ccfa20610217db589307
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.9.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



We redefine our `TrainingArguments`:

In [ ]:
training_args = TrainingArguments(
    "test-clm",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"{model_checkpoint}-wikitext2",
)

Like before, the last two arguments are to setup everything so we can push the model to the [Hub](https://huggingface.co/models) at the end of training. Remove the two of them if you didn't follow the installation steps at the top of the notebook, otherwise you can change the value of `push_to_hub_model_id` to something you would prefer.

Finally, we use a special `data_collator`. The `data_collator` is a function that is responsible of taking the samples and batching them in tensors. In the previous example, we had nothing special to do, so we just used the default for this argument. Here we want to do the random-masking. We could do it as a pre-processing step (like the tokenization) but then the tokens would always be masked the same way at each epoch. By doing this step inside the `data_collator`, we ensure this random masking is done in a new way each time we go over the data.

To do this masking for us, the library provides a `DataCollatorForLanguageModeling`. We can adjust the probability of the masking:

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

Then we just have to pass everything to `Trainer` and begin training:

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 18761
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7038


Epoch,Training Loss,Validation Loss
1,7.091100,7.049844
2,6.905400,6.870386
3,6.856900,6.888713


Saving model checkpoint to test-clm/checkpoint-500
Configuration saved in test-clm/checkpoint-500/config.json
Model weights saved in test-clm/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1000
Configuration saved in test-clm/checkpoint-1000/config.json
Model weights saved in test-clm/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-1500
Configuration saved in test-clm/checkpoint-1500/config.json
Model weights saved in test-clm/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpoint-2000
Configuration saved in test-clm/checkpoint-2000/config.json
Model weights saved in test-clm/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2009
  Batch size = 8
Saving model checkpoint to test-clm/checkpoint-2500
Configuration saved in test-clm/checkpoint-2500/config.json
Model weights saved in test-clm/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test-clm/checkpo

TrainOutput(global_step=7038, training_loss=7.04967836345858, metrics={'train_runtime': 790.1373, 'train_samples_per_second': 71.232, 'train_steps_per_second': 8.907, 'total_flos': 4683068522136576.0, 'train_loss': 7.04967836345858, 'epoch': 3.0})

Like before, we can evaluate our model on the validation set. The perplexity is much lower than for the CLM objective because for the MLM objective, we only have to make predictions for the masked tokens (which represent 15% of the total here) while having access to the rest of the tokens. It's thus an easier task for the model.

In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

***** Running Evaluation *****
  Num examples = 2009
  Batch size = 8


Perplexity: 947.45


The perplexity is still quite high since for this demo we trained on a small dataset for a small number of epochs. For a real LM training, you  would need a larger dataset and more epochs.

You can now upload the result of the training to the Hub, just execute this instruction:

In [ ]:
trainer.push_to_hub()